In [5]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [2]:
def get_song_links(url):
    """Gets song titles and URLs for the lyrics page for a www.songlyrics.com top-songs page."""
    soup = BeautifulSoup(requests.get(url).content)
    trs = soup.find_all('table', class_='tracklist')[0].find_all('tr')
    song_links = {}
    year = url[-4:]
    for row in trs[1:]:
        tds = row.find_all('td')
        artist = tds[1].get_text(strip=True)
        a_tag = tds[2].a
        song_links[(year, artist, a_tag['title'])] = a_tag['href']
    return song_links

In [3]:
def get_lyrics(url):
    """Scrapes the lyrics from a www.songlyrics.com song page."""
    soup = BeautifulSoup(requests.get(url).content, )
    return soup.find_all('p', attrs={'id':'songLyricsDiv'})[0].get_text('\n', strip=True)

In [4]:
# Define variables
base_url = 'http://www.songlyrics.com/news/top-songs/'
start_year = 2011
end_year = 1950

In [9]:
# Dict to hold song titles -> link to lyrics page.
song_links = {}
cur_year = start_year

retries = 0
max_retries = 3
pause_length = 1   # in seconds

# Loop through years and scrape links to lyric pages for each year's top 100 songs.
while cur_year >= end_year:
    print(cur_year)
    url = base_url + str(cur_year)
    try:
        song_links.update(get_song_links(url))
        retries = 0
    except Exception as e:  # This should be ConnectionError
        if retries < max_retries:
            retries += 1
            time.sleep(pause_length)
            cur_year += 1
        else:
            print('Max retries reached.')
            print(str(e))
    cur_year -= 1

2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
1989
1988
1987
1986
1985
1984
1983
1982
1981
1980
1979
1978
1977
1976
1975
1974
1973
1972
1971
1970
1969
1968
1967
1966
1965
1964
1963
1962
1961
1960
1959
1958
1957
1956
1955
1954
1953
1952
1951
1950


NameError: name 'ConnectionError' is not defined

In [15]:
%%time
# Dict to hold song titles -> lyrics
song_lyrics = {}
errors = {}
# Visit the lyric pages for each song in song_links and scrape the lyrics.
for artist_title, url in song_links.iteritems():
    try:
        song_lyrics[artist_title] = get_lyrics(url)
    except Exception as e:
        errors[artist_title] = e

Wall time: 1h 37min 15s


In [16]:
len(song_lyrics)

4936

In [17]:
len(errors)

1157

In [18]:
# Convert tuples to lists.
song_lyrics2 = [[list(k), v] for k,v in song_lyrics.items()]

In [19]:
# Save song lyrics.
json.dump(song_lyrics2, open('data/song_lyrics_2.json', 'wt'), encoding='utf8')

http://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html
TextBlob